<a href="https://colab.research.google.com/github/kahyunp/viva/blob/main/%EC%B5%9C%EC%A2%85)%ED%8F%89%EA%B0%80%EC%9B%90_%ED%8E%98%EC%9D%B4%EC%A7%80%EB%B2%88%ED%98%B8_%EC%9D%BD%EA%B8%B0(%EC%B9%B4%EC%B9%B4%EC%98%A4api).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import glob
#이미지 읽어오기
def cropBottom(file):
    img = cv2.imread(file)
  
    plt.figure(figsize=(15,12))

    #이미지 흑백처리
    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)

    #이미지내의 경계 찾기
    ret, img_th = cv2.threshold(img_blur, 127, 255, cv2.THRESH_BINARY_INV)
    contours, hierachy= cv2.findContours(img_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #경계를 직사각형으로 찾기
    rects = [cv2.boundingRect(each) for each in contours]
    rects=sorted(rects)
    bottom=rects[-1][1]+rects[-1][3]
    margin=10
    img=img[:bottom+margin,:]
    cv2_imshow(img)
    return img, rects
# for rect in rects:
#     print(rect)
#     cv2.circle(img, (rect[0],rect[1]),3,(0,0,255), -1)
#     cv2.circle(img, (rect[0]+rect[2],rect[1]+rect[3]),3,(0,0,255), -1)
#     cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,255,0),3)
#     cv2_imshow(img)   
# print(len(rects))

if __name__ == '__main__' :
    os.chdir("/content/drive/MyDrive/GRADING_Study/kh/header/")
    images=glob.glob('./*.png')
    images.sort()
    for i in range(len(images)):
        filename=images[i]
        print(filename)
        img,rects=cropBottom(filename)
        img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
        ans=[]
        #페이지 1인 경우 글자가 많아서 가능
        if (len(rects)>30):
            print(1)
        else:
            path='/content/drive/MyDrive/GRADING_Study/kh/crophead/'
            name= filename[-6:]
            newfile=path+name
            w=img.shape[1]
            left=img_gray[:,:int(w/4)]
            right=img_gray[:,3*int(w/4):]
            page=cv2.hconcat([left, right])
            cv2_imshow(page)
            cv2.imwrite(newfile,page)
            #ans=findPage(page)
            #resizeImg(page, ans)




    


In [ ]:
import json

import cv2
import requests
import sys
from google.colab.patches import cv2_imshow
LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
 
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})
# def makeBorder(input):

#     im = cv2.imread(input)
#     row, col = im.shape[:2]
#     bottom = im[row-2:row, 0:col]
#     mean = cv2.mean(bottom)[0]

#     bordersize = 400
#     border = cv2.copyMakeBorder(
#         im,
#         top=bordersize,
#         bottom=bordersize,
#         left=bordersize,
#         right=bordersize,
#         borderType=cv2.BORDER_CONSTANT,
#         value=[mean, mean, mean]
#     )
#     return border

#     #cv2_imshow(im)
#     #print("bottom")
#     #cv2_imshow(bottom)
#     print("border added")

def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    image_path, appkey = '/content/drive/MyDrive/GRADING_Study/kh/crophead/12.png', '앱키 여기에 적으면됨'
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr(image_path, appkey).json()
    print("[OCR] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2)))


if __name__ == "__main__":
    main()
  



위의 두개만 쓰면